# Entrega 1
#### ManyBody - Claudia Garcia, Adrià Rojo - Abril 2025

In [46]:
import numpy as np
import numpy.typing as npt
import sympy as sp
import scipy.interpolate as scint
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display, Math

def second_derivative(fun: npt.ArrayLike, h: float) -> npt.ArrayLike:
    first = np.gradient(fun, h)
    return np.gradient(first, h)

def final_properties(psi, step, rs, r2s, interaction):
    psi_dx2 = second_derivative(psi, step)

    local_values = [
        (r2*final*final, #acc radius
            final*dx2, # kin energy
            r2*final*final, # armonic
            r2*(final/r)**4 if r != 0 else 0, # interaction energy
            mu*final*final, # 
            r2/2 + interaction * (final/r)**2 if r != 0 else 0,
            (final/r)**2 if r != 0 else 0
        )
        for r, r2, final, dx2, mu in zip(rs, r2s, psi, psi_dx2, local_mu)
    ]

    local_accumulted_radius, local_kinetic_energy, local_armonic_potential, local_interaction_energy, local_average_chem_potential, particle_potential, density =\
        zip(*local_values)

    squared_radius = sum(local_accumulted_radius)*step
    mean_squared_radius = np.sqrt(squared_radius)
    average_chem_potential = sum(local_average_chem_potential)*step
    kinetic_energy = -sum(local_kinetic_energy)*step/2
    armonic_potential = sum(local_armonic_potential)*step/2
    interaction_energy = sum(local_interaction_energy)*step*interaction/2

    return {
        "r^2": squared_radius,
        "<r^2>": mean_squared_radius,
        "avg mu": average_chem_potential,
        "kin energy": kinetic_energy,
        "armonic pot" : armonic_potential,
        "interac energy": interaction_energy
    }

def initial_values(scattering_length, width, step, atom_numbers, alpha, cvar):
    alpha2 = alpha*alpha
    cvar = 2 * np.power(alpha, 3/2)/ np.power(np.pi, 1/4)
    rs = np.array([i * step for i in range(width)])
    r2s = rs**2
    psi = np.array([cvar*r*np.exp(-0.5*alpha2*r2) for (r, r2) in zip(rs, r2s)])
    return alpha2, cvar, scattering_length*atom_numbers, atom_numbers*scattering_length*scattering_length*scattering_length,\
        rs, r2s, psi
    
def compute_wavelength(psi, iterations, step, interaction, time_step):
    psi_curr = psi
    for i in tqdm(range(iterations)):
        psi_dx2 = second_derivative(psi_curr, step)
        # calculo de energia y mu local
        local_energy_mu = [(-(curr*dx2+r2*curr**2+interaction*r2*(curr/r)**4)/2 if r != 0 else 0, 
                            ((-dx2/curr) + r2 + interaction*(curr/r)**2)/2 if r != 0 else 0)
                        for r, r2, curr, dx2 in zip(rs, r2s, psi_curr, psi_dx2)]
        # deshacer la lista de tuplas (energía y mu de cada distancia) 
        # en una tupla de listas (lista de todas las energías y lista de todas las mus)
        local_energy, local_mu = zip(*local_energy_mu)
        energy = sum(local_energy)*step
        psi_next = [curr*(1 - time_step*mu) for curr, mu in zip(psi_curr, local_mu)]
        normalization_const = np.sqrt(sum(next*next for next in psi_next)*step)
        psi_curr = np.array(psi_next)/normalization_const
    
    return (psi_curr, local_mu)

In [45]:

scattering_length, width, step, atom_numbers, time_step, alpha, iterations = \
    0.00433, 400, 0.015, 10000, 0.0001, 0.8, 40000
    # 0.00433, 700, 0.015, 1000000, 0.00005, 0.3, 70000
    # 0.00433, 600, 0.015, 100000, 0.0001, 0.4, 50000
    # 0.00433, 400, 0.020, 1000, 0.0001, 0.5, 50000
    # 0.00433, 300, 0.020, 100, 0.0001, 0.5, 50000  

alpha2, cvar, interaction, density_param, rs, r2s, psi = \
    initial_values(scattering_length, width, step, atom_numbers, alpha)

psi, local_mu = compute_wavelength(psi, iterations, step, interaction, time_step)

100%|██████████| 40000/40000 [00:19<00:00, 2085.46it/s]
